In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
np.set_printoptions(precision=3, suppress=True)

In [3]:
PER_STOCK_DATA_FOLDER_NAME = 'stock_data'
STOCK_FILE_NAME = "{STOCK_ID}.csv"
REQUIRED_COLS_FOR_TRAINING = ['成交股數','開盤價', '最高價', '最低價', '收盤價']

In [4]:
def read_csv(stock_id):
    file_name = STOCK_FILE_NAME.format(STOCK_ID=stock_id)
    file_path = os.path.join(PER_STOCK_DATA_FOLDER_NAME, file_name)
    with open(file_path) as f:
        df = pd.read_csv(f)
        return df

In [5]:
stock_df_orig = read_csv('1419')

# 資料概觀

In [6]:
stock_df_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3070 entries, 0 to 3069
Data columns (total 8 columns):
Unnamed: 0    3070 non-null int64
證券代號          3070 non-null int64
成交股數          3070 non-null int64
開盤價           3070 non-null object
最高價           3070 non-null object
最低價           3070 non-null object
收盤價           3070 non-null object
日期            3070 non-null int64
dtypes: int64(4), object(4)
memory usage: 192.0+ KB


除了數字外， 開/高/低/收 columns的data type 為 object，可能有非數值的資料，根據raw data，發現當天盤中若沒有成交資料，上述columns的資料會被填入 '--'

# 資料整理

In [7]:
# 列出含有 '--' 的columns
invalid_value_cols = stock_df_orig['開盤價'] == '--'
stock_df_orig.loc[invalid_value_cols].head()

,Unnamed: 0,證券代號,成交股數,開盤價,最高價,最低價,收盤價,日期
2445,186861,1419,11,--,--,--,--,20151105
2657,162457,1419,0,--,--,--,--,20160914
2726,229488,1419,0,--,--,--,--,20161227
2754,23791,1419,336,--,--,--,--,20170214
2758,27747,1419,765,--,--,--,--,20170218


In [8]:
#利用 pd.to_numeric 將整個data frame轉換成numeric type，並將無法轉換的 data cell，填入nan
stock_df_processed = stock_df_orig.apply(pd.to_numeric, errors='coerce')
stock_df_processed.loc[invalid_value_cols].head()

,Unnamed: 0,證券代號,成交股數,開盤價,最高價,最低價,收盤價,日期
2445,186861,1419,11,NaN,NaN,NaN,NaN,20151105
2657,162457,1419,0,NaN,NaN,NaN,NaN,20160914
2726,229488,1419,0,NaN,NaN,NaN,NaN,20161227
2754,23791,1419,336,NaN,NaN,NaN,NaN,20170214
2758,27747,1419,765,NaN,NaN,NaN,NaN,20170218


In [9]:
stock_df_processed = stock_df_processed.dropna(how='any') 
row_with_nan = stock_df_processed.isnull().any(axis=1)
stock_df_processed[row_with_nan]

,Unnamed: 0,證券代號,成交股數,開盤價,最高價,最低價,收盤價,日期


In [10]:
stock_df_processed['成交股數'] = (stock_df_processed['成交股數']/1000).round(0)
stock_df_processed.head()

,Unnamed: 0,證券代號,成交股數,開盤價,最高價,最低價,收盤價,日期
0,69,1419,2435.0,19.45,20.40,19.15,19.80,20080102
1,785,1419,468.0,19.40,19.80,19.20,19.60,20080103
2,1500,1419,1804.0,19.40,20.30,19.20,20.05,20080104
3,2216,1419,1644.0,19.50,20.15,19.05,19.60,20080107
4,2932,1419,1203.0,19.50,20.30,19.40,19.90,20080108


In [15]:
stock_df = stock_df_processed[REQUIRED_COLS_FOR_TRAINING]
stock_df.head()

,成交股數,開盤價,最高價,最低價,收盤價
0,2435.0,19.45,20.40,19.15,19.80
1,468.0,19.40,19.80,19.20,19.60
2,1804.0,19.40,20.30,19.20,20.05
3,1644.0,19.50,20.15,19.05,19.60
4,1203.0,19.50,20.30,19.40,19.90


In [16]:
#資料正規化 
from sklearn.preprocessing import MinMaxScaler
minMaxScaler = MinMaxScaler()
stock_min_max = minMaxScaler.fit_transform(stock_df.as_matrix())
stock_df = pd.DataFrame(stock_min_max, columns=stock_df.columns)

In [17]:
stock_df.head()

,成交股數,開盤價,最高價,最低價,收盤價
0,0.080407,0.210000,0.215642,0.215262,0.209302
1,0.015427,0.208889,0.202235,0.216401,0.204873
2,0.059562,0.208889,0.213408,0.216401,0.214839
3,0.054276,0.211111,0.210056,0.212984,0.204873
4,0.039708,0.211111,0.213408,0.220957,0.211517
